## Finetuning DistilBERT using SQuAD 2.0

This notebook contains the following fine-tuning groups:  Model 3-5

....

The starting-point for code in this file was found in the Medium blog post titled, Question Answering with DistilBERT (https://medium.com/@sabrinaherbst/question-answering-with-distilbert-ba3e178fdf3d). Main differences include:
 - The DistilBERT model was pre-trined using SQuAD 2.0, rather than SQuAD 1.0
  - Exploring traditional split that included unseen test data  (i.e. not validation data)
 - Addition of adhoc dropout rate setting
 - Additional stats for failed results

....

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load libraries

from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertConfig, \
            DistilBertTokenizerFast, AutoTokenizer, BertModel, BertForMaskedLM, BertTokenizerFast, BertConfig
from torch import nn
from pathlib import Path
import torch
import pandas as pd
from typing import Optional
from tqdm.auto import tqdm
from torch.optim import AdamW, RMSprop
import numpy as np

import sys
sys.path.append('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project')
from qa_model import QuestionDistilBERT, SimpleQuestionDistilBERT, ReuseQuestionDistilBERT, Dataset, test_model
from util import eval_test_set, count_parameters, print_test_set_incorrect_predictions, \
            analyze_test_set_statistics, analyze_test_set_performance

# Load tokenizer


In [ ]:
# Load DistilBERT tokenizer, use uncased (lowercase) vocabulary

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Load data_2 (Traditional split)

Note: Dataset loader is the "Variable-Length Trunc" described in the report

In [ ]:
# Get paths for all SQuAD dataset text files in training directory -- data_2, Traditional Split
squad_paths_2 = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')]

# Create training dataset using only SQuAD data, DataLoader with batch size of 8
dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')],
                  natural_question_paths=None,
                  hotpotqa_paths=None, tokenizer=tokenizer)
loader_2 = torch.utils.data.DataLoader(dataset_2, batch_size=8)
print(f"Approximate Training Dataset Size: {len(dataset_2)}")

## load the validation dataset -- used to be labeled as "test", test_dataset changed to val_dataset, test_loader changed to val_loader
val_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/validation_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
val_loader_2 = torch.utils.data.DataLoader(val_dataset_2, batch_size=4)
print(f"Approximate Validation Dataset Size: {len(val_dataset_2)}")

## load the test dataset -- test_dataset and test_loader should not be used during training
test_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/test_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=4)
print(f"Approximate Test Dataset Size: {len(test_dataset_2)}")

Approximate Training Dataset Size: 113000
Approximate Validation Dataset Size: 14000
Approximate Test Dataset Size: 14000


# Model_3

Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split.
 Alter training parameters to 4 epochs with a dropout rate of 0.250

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_3 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_3 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_3 = model_3.distilbert

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_3 = SimpleQuestionDistilBERT(mod_3)
model_3.set_dropout_rate(0.25)
model_3.to(device)

# Verify the dropout rates for each layer
for name, module in model_3.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.25
distilbert.transformer.layer.0.attention.dropout: 0.25
distilbert.transformer.layer.0.ffn.dropout: 0.25
distilbert.transformer.layer.1.attention.dropout: 0.25
distilbert.transformer.layer.1.ffn.dropout: 0.25
distilbert.transformer.layer.2.attention.dropout: 0.25
distilbert.transformer.layer.2.ffn.dropout: 0.25
distilbert.transformer.layer.3.attention.dropout: 0.25
distilbert.transformer.layer.3.ffn.dropout: 0.25
distilbert.transformer.layer.4.attention.dropout: 0.25
distilbert.transformer.layer.4.ffn.dropout: 0.25
distilbert.transformer.layer.5.attention.dropout: 0.25
distilbert.transformer.layer.5.ffn.dropout: 0.25
dropout: 0.25


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_3.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = RMSprop(model_3.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 4 epochs
# Validation data used to evaluate performance during training

epochs = 4

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_3.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_3(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_3.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_3(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.6182086976777135


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.200955101034471


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.1387857078313828


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 0.7712526111559267


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.322631913753426


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_3.state_dict(), "simple_distilbert_qa_data_3.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_3 = SimpleQuestionDistilBERT(mod_3)

# Load previously saved model parameters from disk
model_3.load_state_dict(torch.load("simple_distilbert_qa_data_3.model"))

<ipython-input-11-3ceeaff87801>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_3.load_state_dict(torch.load("simple_distilbert_qa_data_3.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_3, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:10<00:00, 49.64it/s]

Mean EM:  0.6245714285714286
Mean F-1:  0.7017227926338852


# Model_4

Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split.
 Alter training parameters to 4 epochs and dropout rate of 0.225

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_4 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_4 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_4 = model_4.distilbert

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_4 = SimpleQuestionDistilBERT(mod_4)
model_4.set_dropout_rate(0.225)
model_4.to(device)

# Verify the dropout rates for each layer
for name, module in model_4.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.225
distilbert.transformer.layer.0.attention.dropout: 0.225
distilbert.transformer.layer.0.ffn.dropout: 0.225
distilbert.transformer.layer.1.attention.dropout: 0.225
distilbert.transformer.layer.1.ffn.dropout: 0.225
distilbert.transformer.layer.2.attention.dropout: 0.225
distilbert.transformer.layer.2.ffn.dropout: 0.225
distilbert.transformer.layer.3.attention.dropout: 0.225
distilbert.transformer.layer.3.ffn.dropout: 0.225
distilbert.transformer.layer.4.attention.dropout: 0.225
distilbert.transformer.layer.4.ffn.dropout: 0.225
distilbert.transformer.layer.5.attention.dropout: 0.225
distilbert.transformer.layer.5.ffn.dropout: 0.225
dropout: 0.225


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_4.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = RMSprop(model_4.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 4 epochs
# Validation data used to evaluate performance during training

epochs = 4

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_4.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_4(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_4.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_4(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.596969238827714


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.207909920845713


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.201102013158878


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 0.7400725294544106


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.24293295129801


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_4.state_dict(), "simple_distilbert_qa_data_4.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_4 = SimpleQuestionDistilBERT(mod_4)

# Load previously saved model parameters from disk
model_4.load_state_dict(torch.load("simple_distilbert_qa_data_4.model"))

<ipython-input-20-ddfe4feeb9b4>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_4.load_state_dict(torch.load("simple_distilbert_qa_data_4.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_4, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:04<00:00, 54.61it/s]

Mean EM:  0.6384285714285715
Mean F-1:  0.7116972048056921


# Model_5
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split; Alter training parameters to 4 epochs with dropout rate of 0.18

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_5 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_5 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_5 = model_5.distilbert

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_5 = SimpleQuestionDistilBERT(mod_5)
model_5.set_dropout_rate(0.18)
model_5.to(device)

# Verify the dropout rates for each layer
for name, module in model_5.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.18
distilbert.transformer.layer.0.attention.dropout: 0.18
distilbert.transformer.layer.0.ffn.dropout: 0.18
distilbert.transformer.layer.1.attention.dropout: 0.18
distilbert.transformer.layer.1.ffn.dropout: 0.18
distilbert.transformer.layer.2.attention.dropout: 0.18
distilbert.transformer.layer.2.ffn.dropout: 0.18
distilbert.transformer.layer.3.attention.dropout: 0.18
distilbert.transformer.layer.3.ffn.dropout: 0.18
distilbert.transformer.layer.4.attention.dropout: 0.18
distilbert.transformer.layer.4.ffn.dropout: 0.18
distilbert.transformer.layer.5.attention.dropout: 0.18
distilbert.transformer.layer.5.ffn.dropout: 0.18
dropout: 0.18


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_5.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = RMSprop(model_5.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 3 epochs
# Validation data used to evaluate performance during training

epochs = 3

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_5.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_5(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_5.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_5(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.5136810018805276


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.1621834735529764


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.114624977353428


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 0.8068674535202769


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.2164758030270626


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_5.state_dict(), "simple_distilbert_qa_data_5.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_5 = SimpleQuestionDistilBERT(mod_5)

# Load previously saved model parameters from disk
model_5.load_state_dict(torch.load("simple_distilbert_qa_data_5.model"))

<ipython-input-28-6521397bc89a>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_5.load_state_dict(torch.load("simple_distilbert_qa_data_5.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_5, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:04<00:00, 54.41it/s]

Mean EM:  0.6438571428571429
Mean F-1:  0.7137698566055884
